In [1]:
import pandas as pd

In [2]:
# load data
# requires data to be in the parent directory
postings             = pd.read_csv('../data/postings.csv')
benefits             = pd.read_csv('../data/jobs/benefits.csv')
job_industries       = pd.read_csv('../data/jobs/job_industries.csv')
job_skills           = pd.read_csv('../data/jobs/job_skills.csv')
salaries             = pd.read_csv('../data/jobs/salaries.csv')
companies            = pd.read_csv('../data/companies/companies.csv')
company_industries   = pd.read_csv('../data/companies/company_industries.csv')
company_specialities = pd.read_csv('../data/companies/company_specialities.csv')
employee_counts      = pd.read_csv('../data/companies/employee_counts.csv')

In [49]:
postings             = pd.read_csv('../data/postings.csv')

postings = postings[['job_id', 'company_name', 'title', 'location', 'company_id', \
                     'original_listed_time', 'formatted_experience_level', \
                     'currency', 'normalized_salary', 'zip_code']]

In [52]:
# change original_listed_time to datetime
postings['original_listed_time'] = pd.to_datetime(postings['original_listed_time'], unit='ms', errors='coerce')
postings = postings.sort_values(by='original_listed_time', ascending=True)

# extract state data
stateAbbrToName = {
  "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
  "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
  "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
  "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
  "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
  "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
  "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
  "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
  "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
  "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
  "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
  "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
  "WI": "Wisconsin", "WY": "Wyoming", "DC": "District of Columbia"
}
postings['state'] = postings['location'].str.extract(r',\s*([A-Z]{2})')
postings['state_full'] = postings['state'].map(stateAbbrToName)

# # remove NAN data for normalized_salary
# postings = postings[postings['normalized_salary'].notna()]

print(len(postings))
postings

123849


,job_id,company_name,title,location,company_id,original_listed_time,formatted_experience_level,currency,normalized_salary,zip_code,state,state_full
322,3778723781,Northeast Georgia Health System,Registered Nurse RN - Medical ICU MICU - PT Ni...,"Gainesville, GA",120000.0,2023-12-05 21:08:53,Mid-Senior level,NaN,NaN,30501.0,GA,Georgia
326,3782349432,NaN,Commercial Appraiser Manager,"Palatka, FL",NaN,2023-12-08 15:47:14,NaN,NaN,NaN,32177.0,FL,Florida
345,3789402808,SOLA Pharmaceuticals,Customer Service Specialist,"Baton Rouge, LA",10617084.0,2023-12-21 18:49:15,NaN,NaN,NaN,70801.0,LA,Louisiana
358,3795156068,Georgia Tech Research Institute,"Field Office ISSM - Open Rank-RS-Albuquerque, NM","Albuquerque, NM",3557.0,2024-01-05 20:18:41,Entry level,NaN,NaN,87101.0,NM,New Mexico
357,3795149962,Georgia Tech Research Institute,"Field Office ISSM - Open Rank-RS-Albuquerque, NM","Albuquerque, NM",3557.0,2024-01-05 20:19:04,Entry level,NaN,NaN,87101.0,NM,New Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...
72687,3902881620,NaN,Member Service Representative,"McCook, NE",NaN,2024-04-20 00:26:07,NaN,NaN,NaN,69001.0,NE,Nebraska
94872,3904778756,CyberApt Recruitment,Manager Security Architecture,"Texas, United States",26489299.0,2024-04-20 00:26:08,NaN,USD,235000.0,NaN,NaN,NaN
72649,3902880673,"Motus Recruiting and Staffing, Inc.",Senior Systems Administrator,"Vancouver, WA",64379.0,2024-04-20 00:26:28,Mid-Senior level,USD,130000.0,98660.0,WA,Washington
123837,3906265414,"TalentBurst, an Inc 5000 company",Contract Administrator,"Irvine, CA",122451.0,2024-04-20 00:26:30,Mid-Senior level,USD,83200.0,92602.0,CA,California


In [75]:
import calendar

# Extract year-month and days in that month
postings['year_month_date'] = postings['original_listed_time'].dt.to_period('D').astype(str)
postings['year'] = postings['original_listed_time'].dt.year
postings['month'] = postings['original_listed_time'].dt.month
postings['date'] = postings['original_listed_time'].dt.month

# Group by state and year-month
grouped = postings.groupby(['state_full', 'year_month_date', 'year', 'month'])

# Aggregate
postings_groped = grouped.agg(
    job_count=('job_id', 'count'),
    median_salary=('normalized_salary', 'median')
).reset_index()

# sort according to state_full
postings_groped = postings_groped.sort_values(by=['state_full', 'year_month_date'])

# fill nan median_salary with -1
postings_groped['median_salary'] = postings_groped['median_salary'].fillna(-1)

postings_groped.head()

,state_full,year_month_date,year,month,job_count,median_salary
0,Alabama,2024-03-12,2024,3,2,-1.0
1,Alabama,2024-03-21,2024,3,1,-1.0
2,Alabama,2024-03-23,2024,3,1,-1.0
3,Alabama,2024-03-24,2024,3,8,-1.0
4,Alabama,2024-03-25,2024,3,1,375000.0


In [73]:
result = {}
for state, group in postings_groped.groupby('state_full'):
    result[state] = [
        {
            "date": row['year_month_date'],
            "job_count": row['job_count'],
            "median_salary": row['median_salary']
        }
        for _, row in group.iterrows()
        if pd.notnull(row['median_salary']) or row['job_count'] > 0
    ]

import json
with open("../data_processed/state_time_series.json", "w") as f:
    json.dump(result, f, indent=2)